In [58]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle


In [59]:
df= pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [60]:
## Drop unnecessary columns
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [61]:
print(df.value_counts('Geography'))
print(df.value_counts('Gender'))

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64
Gender
Male      5457
Female    4543
Name: count, dtype: int64


In [62]:
###Encoding categorical variables
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])
df

with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)

In [63]:
###Onehot encode Geography
onehot_enc=OneHotEncoder()
geo_enccoder = onehot_enc.fit_transform(df[['Geography']]).toarray()

with open('onehot_encoder.pkl', 'wb') as f:
    pickle.dump(onehot_enc, f)

geo_enccoder


array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], shape=(10000, 3))

In [64]:
onehot_enc.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [65]:
geo_df=pd.DataFrame(geo_enccoder,columns=onehot_enc.get_feature_names_out(['Geography']))
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [66]:
###Combining columns
df = pd.concat([df.drop('Geography',axis=1), geo_df], axis=1)
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [67]:
##Divide the data
X= df.drop('Exited',axis=1)
y = df['Exited']

##Split the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Scaling
scaler= StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)

In [68]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]], shape=(8000, 12))

In [69]:
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler,f)

In [71]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [72]:
model = Sequential([
    Dense(64, activation='relu',input_shape=(X_train.shape[1],)), ###First hidden layer Connected with input layer
    Dense(32,activation='relu'),  ###Second hidden layer
    Dense(1,activation='sigmoid')  ###Output layer
])
model.summary()

d:\ANN_Classification\ann_env\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [73]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.001)

In [74]:
##Complie the model 
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy']) 

In [75]:
##TensorBoard setup
import os
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Create the directory if it doesn't exist
os.makedirs(log_dir, exist_ok=True)
print(f"TensorBoard logs will be saved to: {log_dir}")

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

TensorBoard logs will be saved to: logs/fit/20250827-193955


In [76]:
### Set up early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [77]:
## Train the model
history = model.fit(
    X_train, y_train,validation_data=(X_test, y_test),epochs=100,
    callbacks=[tensorboard_callback, early_stopping]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8071 - loss: 0.4548 - val_accuracy: 0.8360 - val_loss: 0.3883
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8071 - loss: 0.4548 - val_accuracy: 0.8360 - val_loss: 0.3883
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8429 - loss: 0.3773 - val_accuracy: 0.8610 - val_loss: 0.3518
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8429 - loss: 0.3773 - val_accuracy: 0.8610 - val_loss: 0.3518
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8562 - loss: 0.3520 - val_accuracy: 0.8525 - val_loss: 0.3559
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8562 - loss: 0.3520 - val_accuracy: 0.8525 - val_loss: 0.3559
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8589 - loss: 0.3436 - val_accuracy: 0.8600 - val_loss: 0.3451
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8589 - loss: 0.3436 - val_accu

In [78]:
model.save('model.h5')

In [79]:
##Load the tenserboard
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [101]:
# Check if logs directory exists and list contents
import os
if os.path.exists('logs'):
    print("Available log directories:")
    for item in os.listdir('logs'):
        if item.startswith('fit'):
            print(f"  - logs/{item}")
    
    # Start TensorBoard with the logs directory
    %tensorboard --logdir logs
else:
    print("No logs directory found. Please train the model first.")

Available log directories:
  - logs/fit
  - logs/fit20250827-153006


Reusing TensorBoard on port 6007 (pid 27912), started 4:11:42 ago. (Use '!kill 27912' to kill it.)

In [80]:
from tensorflow.keras.models import load_model

##Load the trained model, scaler,onehot
model = load_model('model.h5')

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('onehot_encoder.pkl', 'rb') as f:
    onehot_enc = pickle.load(f)

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)


In [91]:
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':35,
    'Tenure':5,
    'Balance':600000,
    'NumOfProducts':3,
    'HasCrCard':0,
    'IsActiveMember':1,
    'EstimatedSalary':500000,}

In [92]:
##one hot encode
geo_encode = onehot_enc.transform([[input_data['Geography']]]).toarray()
geo_encode_df = pd.DataFrame(geo_encode,columns=onehot_enc.get_feature_names_out(['Geography']))
geo_encode_df

d:\ANN_Classification\ann_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [93]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,35,5,600000,3,0,1,500000


In [94]:
# Remove Geography column before encoding Gender

input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,35,5,600000,3,0,1,500000


In [95]:
## Concat processed data
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encode_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,35,5,600000,3,0,1,500000,1.0,0.0,0.0


In [96]:
##Scale the data
input_scale = scaler.transform(input_df)
input_scale

array([[-5.35985161e-01,  9.13247552e-01, -3.70568589e-01,
        -1.34471949e-03,  8.38812313e+00,  2.53355998e+00,
        -1.54035103e+00,  9.74816989e-01,  6.94716943e+00,
         1.00150113e+00, -5.79467227e-01, -5.76388018e-01]])

In [97]:
##Predict 
prediction = model.predict(input_scale)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


array([[0.7551177]], dtype=float32)

In [98]:
prediction_prob = prediction[0][0]

In [99]:
prediction_prob

np.float32(0.7551177)

In [100]:
if prediction_prob > 0.5:
    print("Positive class")
else:
    print("Negative class")

Positive class
